# 6. Ufuncs

## Exercise  (`np.einsum`)

*Exercise from [100 numpy exercises](https://github.com/rougier/numpy-100)*

Use `np.einsum` to calculate the **diagonal of a dot product** of two matrices (`np.diag(np.dot(A, B))`).

```
A = np.arange(6).reshape(3, 2)
B = np.ones((2, 3))
np.einsum('your signature goes here', A, B)
```

Then, test your solution on stacked arrays:

```
A = np.arange(12).reshape(2, 3, 2)
B = np.ones((2, 3))
```

In [21]:
A = np.arange(12).reshape(2, 3, 2)
B = np.ones((2, 3))
np.einsum('...ij,...ji->...i', A, B)

array([[  1.,   5.,   9.],
       [ 13.,  17.,  21.]])


# 7. Extending NumPy

### Exercise

Take the following function calculating logit and turn it into a ufunc using the above example.

```cython

cdef extern from "math.h":
    double log "log" (double) nogil
    
import cython

@cython.cdivision(True)
cdef double logit_double(double p) nogil:
    p = p/(1-p);
    p = log(p);
    return p
```

In [13]:
%load_ext cython

The cython extension is already loaded. To reload it, use:
  %reload_ext cython


In [14]:
%%cython

# The elementwise function

cdef extern from "math.h":
    double log "log" (double) nogil
    
import cython

@cython.cdivision(True)
cdef double logit_double(double p) nogil:
    p = p/(1-p);
    p = log(p);
    return p


# Required module initialization
# ------------------------------

cimport numpy as np
np.import_array()
np.import_ufunc()

# The actual ufunc declaration
# ----------------------------

cdef np.PyUFuncGenericFunction loop_func[1]
cdef char input_output_types[2]
cdef void *elementwise_funcs[1]

loop_func[0] = np.PyUFunc_d_d # generic function to implement looping

input_output_types[0] = np.NPY_DOUBLE
input_output_types[1] = np.NPY_DOUBLE


elementwise_funcs[0] = <void*>logit_double

logit = np.PyUFunc_FromFuncAndData(
    loop_func,
    elementwise_funcs,
    input_output_types,
    1, # number of supported input types
    1, # number of input args
    1, # number of output args
    0, # `identity` element, never mind this
    "logit", # function name
    "computes logit", # docstring
    0 # unused
    )


In [23]:
import numpy as np
a = np.array([0.9, 0.3], dtype=np.double)
logit(a)

array([ 2.19722458, -0.84729786])